In [384]:
import json
from collections import Counter

from cogent3 import PhyloNode
import numpy as np
from scipy import special
from sklearn.cluster import SpectralClustering
import pandas as pd
import networkx as nx

In [205]:
with open('../data/iq-tree-third.json') as fh:
    iq_tree_third = json.load(fh)

In [402]:
with open('white-dwarf-third.json') as fh:
    third_triples, third_losses, third_params = json.load(fh)

In [207]:
with open('neutron-second.json') as fh:
    second_triples, second_losses, second_params = json.load(fh)

In [409]:
def cherry_weights(lens, ls, N):
    ls = N*ls
    delta = ls - ls.min()
    weights = np.exp(-delta)
    weights = weights/weights.sum()
    return lens*ls
#    return weights
    h = (-weights*np.log(weights))[weights != 0.].sum()
    return lens*ls/(h + 1)
#     print(ls)
#     print(weights)
#     print((ls == ls.min())/(h + 1))
#     print()
#    return (ls == ls.min())/(h + 1)
    return weights/(h + 1)  # this was my favourite
    if h < 1.2:
        return weights
        return (ls == ls.min())/(h+1)
    return np.zeros(3, dtype=bool)

def get_edges(triples, lengths, losses):
    edges = Counter()
    for ln, ls, (names, F) in zip(lengths, losses, triples):
        weights = cherry_weights(ln, ls, F.sum())
        for name, weight in zip(names, weights):
            edges[frozenset(names) - {name}] += weight
    # for edge in edges:
    #   edges[edge] = np.exp(edges[edge])
    return edges

def pulsar(triples, learning_rate=0.01, steps=3000, verbose=False):
    losses, parameters = fit_triples(triples, learning_rate=learning_rate,
                         steps=steps, verbose=verbose)
    lengths = [np.exp(p[:,2,2]) for p in parameters]
    tree = pulsar_tree(triples, lengths, verbose=verbose)
    return tree

def edges_to_graph(edges):
    G = nx.Graph()
    for edge, weight in edges.items():
        G.add_edge(*edge, weight=weight)
    return G

def normalised_cut(edges, verbose=False):
    tips = np.unique([t for p in edges.keys() for t in p])
    affinity = np.zeros((len(tips),)*2)
    for i, tipi in enumerate(tips):
        for j, tipj in enumerate(tips):
            if i == j:
                break
            affinity[i, j] = edges[frozenset((tipi, tipj))]
    affinity += affinity.T
    sc = SpectralClustering(2, affinity='precomputed',  # random_state=0,
                            assign_labels='discretize')
    ix = sc.fit_predict(affinity).astype(bool)
    partition = list(tips[ix]), list(tips[np.logical_not(ix)])
    if verbose:
        G = edges_to_graph(edges)
        cut_value = nx.cut_size(G, partition[0], weight='weight')
        print(f'Cut value: {cut_value}, Partition:\n{partition}')
    return partition
    
def min_cut(edges, verbose=False):
    G = edges_to_graph(edges)
    cut_value, partition = nx.stoer_wagner(G)
    if verbose:
        print(f'Cut value: {cut_value}, Partition:\n{partition}')
    return partition

def pulsar_tree(triples, lengths, losses, verbose=False):
    edges = get_edges(triples, lengths, losses)
    if verbose:
        print('Graph:')
        for edge, weight in edges.items():
            print(edge, weight)
    partition = normalised_cut(edges, verbose)
    assert len(partition) == 2, 'polytomy detected. bailing'
    this_node = PhyloNode()
    for part in partition:
        if len(part) <= 1:
            this_node.append(PhyloNode(part.pop()))
            continue
        elif len(part) == 2:
            child = PhyloNode()
            for grandchild in part:
                child.append(PhyloNode(grandchild))
            this_node.append(child)
            continue
    
        part = set(part)
        part_lengths = []
        part_losses = []
        part_triples = []
        for ln, ls, (names, F) in zip(lengths, losses, triples):
            if set(names) <= part:
                part_lengths.append(ln)
                part_losses.append(ls)
                part_triples.append((names, F))
        this_node.append(pulsar_tree(part_triples, part_lengths, part_losses, verbose=verbose))
    return this_node

In [363]:
def get_trees(all_triples, all_lengths, all_losses):
    trees = []
    for i in range(0, len(all_losses), 4):
        tree = pulsar_tree(all_triples[i:i+4], all_lengths[i:i+4], all_losses[i:i+4])
        trees.append(tree)
    return trees

In [410]:
all_triples = [(names, np.array(F)) for names, F in third_triples]
all_parameters = np.array(third_params)
all_lengths = all_lengths = [np.exp(p[:,1,2]) for p in all_parameters]
all_losses = np.array(third_losses)

In [411]:
trees = get_trees(all_triples, all_lengths, all_losses)

In [412]:
ghb_siblings = Counter()
ordered_siblings = []
for tree in trees:
    tree = tree.unrooted()
    for sibling in tree.get_node_matching_name('Greater horseshoe bat').parent.children:
        if sibling.name in ('Microbat', 'Pig', 'Horse'):
            ghb_siblings[sibling.name] += 1
            ordered_siblings.append(sibling.name)

In [413]:
ghb_siblings

Counter({'Microbat': 471, 'Pig': 206, 'Horse': 201})

In [414]:
ghb_siblings['Microbat']/len(trees)

0.5364464692482915

In [377]:
siblings = list(ghb_siblings.keys())
contingency = pd.DataFrame(np.zeros((3,3)), index=siblings, columns=siblings, dtype=int)
for iqt, neu in zip(iq_tree_third, ordered_siblings):
    contingency[iqt][neu] += 1

In [378]:
contingency

,Microbat,Pig,Horse
Microbat,409,26,35
Pig,129,47,22
Horse,148,25,37


In [371]:
for i in range(0, len(all_lengths), 4):
    print(iq_tree_third[i//4], ordered_siblings[i//4])
    print('\n'.join(map(str, zip([n for n, _ in all_triples[i:i+4]], all_lengths[i:i+4], all_losses[i:i+4]))))
    edges = get_edges(all_triples[i:i+4], all_lengths[i:i+4], all_losses[i:i+4])
    tips = ['Greater horseshoe bat', 'Microbat', 'Horse', 'Pig']
    affinity = np.zeros((len(tips),)*2)
    for i, tipi in enumerate(tips):
        for j, tipj in enumerate(tips):
            if i == j:
                break
            affinity[i, j] = edges[frozenset((tipi, tipj))]
    affinity += affinity.T
    affinity = pd.DataFrame(affinity, index=tips, columns=tips)
    print(affinity)
    print()

Microbat Horse
(['Greater horseshoe bat', 'Microbat', 'Pig'], array([0.19821926, 0.07906633, 0.00196951]), array([0.00443543, 0.00423496, 0.00491889]))
(['Greater horseshoe bat', 'Microbat', 'Horse'], array([0.11762151, 0.04747485, 0.02076596]), array([0.00331525, 0.00305374, 0.00326237]))
(['Greater horseshoe bat', 'Pig', 'Horse'], array([0.09926102, 0.04328114, 0.16752031]), array([0.0039668 , 0.00472621, 0.00463798]))
(['Microbat', 'Pig', 'Horse'], array([0.1073656 , 0.00229436, 0.14175644]), array([0.0048999 , 0.00526641, 0.00573439]))
                       Greater horseshoe bat  Microbat     Horse       Pig
Greater horseshoe bat               0.000000  0.025321  0.114297  0.363558
Microbat                            0.025321  0.000000  0.131463  0.553309
Horse                               0.114297  0.131463  0.000000  0.300784
Pig                                 0.363558  0.553309  0.300784  0.000000

Microbat Horse
(['Greater horseshoe bat', 'Microbat', 'Pig'], array([0.1701946

                       Greater horseshoe bat  Microbat     Horse       Pig
Greater horseshoe bat               0.000000  0.123922  0.258535  0.889890
Microbat                            0.123922  0.000000  0.368593  0.554049
Horse                               0.258535  0.368593  0.000000  0.340903
Pig                                 0.889890  0.554049  0.340903  0.000000

Microbat Pig
(['Pig', 'Horse', 'Microbat'], array([0.15139396, 0.11267459, 0.08615408]), array([0.0038391 , 0.00375395, 0.00400672]))
(['Pig', 'Horse', 'Greater horseshoe bat'], array([0.1130115 , 0.09885622, 0.0970483 ]), array([0.00422504, 0.0040694 , 0.00424071]))
(['Pig', 'Microbat', 'Greater horseshoe bat'], array([0.07503722, 0.14655074, 0.11847616]), array([0.00475628, 0.00486768, 0.00495516]))
(['Horse', 'Microbat', 'Greater horseshoe bat'], array([0.07545824, 0.13520215, 0.11108208]), array([0.00398994, 0.00428069, 0.00434388]))
                       Greater horseshoe bat  Microbat     Horse       Pig
Great

                       Greater horseshoe bat  Microbat     Horse       Pig
Greater horseshoe bat               0.000000  0.437206  0.519544  0.371370
Microbat                            0.437206  0.000000  0.954821  0.888172
Horse                               0.519544  0.954821  0.000000  0.548299
Pig                                 0.371370  0.888172  0.548299  0.000000

Microbat Pig
(['Greater horseshoe bat', 'Microbat', 'Pig'], array([0.71345939, 0.41559071, 0.15862463]), array([0.00745826, 0.00743286, 0.00739288]))
(['Greater horseshoe bat', 'Microbat', 'Horse'], array([0.84586822, 0.38106476, 0.14935483]), array([0.00826412, 0.0078411 , 0.00839622]))
(['Greater horseshoe bat', 'Pig', 'Horse'], array([0.49422113, 0.40805041, 0.36928322]), array([0.00975704, 0.00931083, 0.00916501]))
(['Microbat', 'Pig', 'Horse'], array([0.46635594, 0.2240392 , 0.27762496]), array([0.00806358, 0.00887237, 0.00720334]))
                       Greater horseshoe bat  Microbat     Horse       Pig
Great

                       Greater horseshoe bat  Microbat     Horse       Pig
Greater horseshoe bat               0.000000  0.551363  0.362852  0.470763
Microbat                            0.551363  0.000000  0.620342  0.598425
Horse                               0.362852  0.620342  0.000000  0.559407
Pig                                 0.470763  0.598425  0.559407  0.000000

Microbat Microbat
(['Horse', 'Pig', 'Greater horseshoe bat'], array([0.14736153, 0.14148111, 1.09449094]), array([0.00798298, 0.00664771, 0.00769048]))
(['Horse', 'Pig', 'Microbat'], array([0.26792564, 0.2145097 , 1.13267701]), array([0.01009695, 0.00445707, 0.00896665]))
(['Horse', 'Greater horseshoe bat', 'Microbat'], array([0.46130578, 0.20793508, 0.06208293]), array([0.00635596, 0.00608534, 0.00621107]))
(['Pig', 'Greater horseshoe bat', 'Microbat'], array([0.25335653, 1.28055912, 0.21689643]), array([0.00482059, 0.00855504, 0.00835601]))
                       Greater horseshoe bat   Microbat      Horse        P

Pig                                 0.100075  0.064015  0.052230  0.000000

Horse Pig
(['Greater horseshoe bat', 'Pig', 'Microbat'], array([0.25848141, 0.11659251, 0.37597314]), array([0.00780071, 0.0078175 , 0.00757318]))
(['Greater horseshoe bat', 'Pig', 'Horse'], array([0.24026278, 0.04216196, 0.33565307]), array([0.00925349, 0.00890518, 0.0086306 ]))
(['Greater horseshoe bat', 'Microbat', 'Horse'], array([0.23175861, 0.09490095, 0.19756085]), array([0.01021006, 0.0098548 , 0.01030943]))
(['Pig', 'Microbat', 'Horse'], array([0.19407492, 0.36350966, 0.15803833]), array([0.0105174 , 0.00982168, 0.01000819]))
                       Greater horseshoe bat  Microbat     Horse       Pig
Greater horseshoe bat               0.000000  0.919839  0.408935  1.792190
Microbat                            0.919839  0.000000  1.375119  1.122581
Horse                               0.408935  1.375119  0.000000  1.807586
Pig                                 1.792190  1.122581  1.807586  0.000000

Microba

...

                       Greater horseshoe bat  Microbat     Horse       Pig
Greater horseshoe bat               0.000000  0.160251  1.405201  1.247170
Microbat                            0.160251  0.000000  1.173247  0.599127
Horse                               1.405201  1.173247  0.000000  1.035553
Pig                                 1.247170  0.599127  1.035553  0.000000

Microbat Pig
(['Microbat', 'Greater horseshoe bat', 'Horse'], array([0.16990468, 0.15824188, 0.19645192]), array([0.00969433, 0.00994107, 0.00976263]))
(['Microbat', 'Greater horseshoe bat', 'Pig'], array([0.17503739, 0.12649375, 0.20416405]), array([0.0096938 , 0.00954562, 0.00956451]))
(['Microbat', 'Horse', 'Pig'], array([0.08568085, 0.19805623, 0.21059222]), array([0.00980938, 0.00964847, 0.00983702]))
(['Greater horseshoe bat', 'Horse', 'Pig'], array([0.13922965, 0.24890401, 0.14141259]), array([0.01008528, 0.00982096, 0.01008058]))
                       Greater horseshoe bat  Microbat     Horse       Pig
Great

                       Greater horseshoe bat  Microbat     Horse       Pig
Greater horseshoe bat               0.000000  0.456145  0.343368  0.752464
Microbat                            0.456145  0.000000  0.508120  0.850345
Horse                               0.343368  0.508120  0.000000  0.440724
Pig                                 0.752464  0.850345  0.440724  0.000000

Microbat Pig
(['Microbat', 'Pig', 'Greater horseshoe bat'], array([0.24766355, 0.49076136, 0.00092242]), array([0.01315011, 0.01307628, 0.01472779]))
(['Microbat', 'Pig', 'Horse'], array([0.1836363 , 0.62001801, 0.08354022]), array([0.00965709, 0.00917608, 0.00963476]))
(['Microbat', 'Greater horseshoe bat', 'Horse'], array([0.13050997, 0.57840936, 0.21041623]), array([0.01215215, 0.01208653, 0.01121707]))
(['Pig', 'Greater horseshoe bat', 'Horse'], array([0.17135597, 0.00160015, 0.24553033]), array([0.01108993, 0.01141843, 0.00917234]))
                       Greater horseshoe bat  Microbat     Horse       Pig
Great

                       Greater horseshoe bat  Microbat     Horse       Pig
Greater horseshoe bat               0.000000  0.481711  0.148223  0.355112
Microbat                            0.481711  0.000000  0.538538  1.152886
Horse                               0.148223  0.538538  0.000000  0.199331
Pig                                 0.355112  1.152886  0.199331  0.000000

Pig Horse
(['Microbat', 'Greater horseshoe bat', 'Horse'], array([0.105385 , 0.9798929, 0.0262453]), array([0.01319707, 0.01302687, 0.01275831]))
(['Microbat', 'Greater horseshoe bat', 'Pig'], array([0.30884915, 1.02581378, 0.01990839]), array([0.01203898, 0.01329173, 0.01233771]))
(['Microbat', 'Horse', 'Pig'], array([0.18398097, 1.13266525, 0.02694202]), array([0.012743  , 0.01277206, 0.01135474]))
(['Greater horseshoe bat', 'Horse', 'Pig'], array([0.31446173, 0.15763205, 0.13412103]), array([0.00872592, 0.00895594, 0.00898375]))
                       Greater horseshoe bat   Microbat     Horse        Pig
Greater h

Pig                                 1.827665  1.213094  1.774626  0.000000

Microbat Horse
(['Pig', 'Microbat', 'Greater horseshoe bat'], array([0.09929702, 0.32232844, 0.21391743]), array([0.0064246 , 0.00759003, 0.00708105]))
(['Pig', 'Microbat', 'Horse'], array([0.06148612, 0.19726442, 0.26730504]), array([0.00905433, 0.01063728, 0.00901317]))
(['Pig', 'Greater horseshoe bat', 'Horse'], array([0.22090903, 0.1803817 , 0.1868946 ]), array([0.00836799, 0.00839134, 0.00712529]))
(['Microbat', 'Greater horseshoe bat', 'Horse'], array([0.32448768, 0.20203052, 0.09931094]), array([0.00919974, 0.00944388, 0.00783502]))
                       Greater horseshoe bat  Microbat     Horse       Pig
Greater horseshoe bat               0.000000  0.443223  1.512969  1.182564
Microbat                            0.443223  0.000000  0.771441  1.228220
Horse                               1.512969  0.771441  0.000000  1.130556
Pig                                 1.182564  1.228220  1.130556  0.000000

Mi

Pig                                 4.083294  2.389999  1.297997  0.000000

Microbat Pig
(['Horse', 'Microbat', 'Greater horseshoe bat'], array([0.40366799, 0.02361221, 0.67278893]), array([0.00804062, 0.00867819, 0.00865155]))
(['Horse', 'Microbat', 'Pig'], array([0.36714154, 0.02288809, 0.76101324]), array([0.00662181, 0.00711282, 0.00714458]))
(['Horse', 'Greater horseshoe bat', 'Pig'], array([0.34186419, 0.10408884, 0.52566497]), array([0.00482448, 0.00570874, 0.00579491]))
(['Microbat', 'Greater horseshoe bat', 'Pig'], array([0.27537855, 0.46537807, 0.29830599]), array([0.0070604 , 0.00711898, 0.00705446]))
                       Greater horseshoe bat  Microbat     Horse       Pig
Greater horseshoe bat               0.000000  1.770892  1.076112  1.189482
Microbat                            1.770892  0.000000  3.726329  1.901316
Horse                               1.076112  3.726329  0.000000  0.250572
Pig                                 1.189482  1.901316  0.250572  0.000000

Micr

In [221]:
s_triples = [(names, np.array(F)) for names, F in second_triples]
s_parameters = np.array(second_params)
s_lengths = all_lengths = [np.exp(p[:,2,2]) for p in s_parameters]
s_losses = np.array(second_losses)

In [241]:
trees = get_trees(s_triples, s_lengths, s_losses)

In [242]:
ghb_siblings = Counter()
s_siblings = []
for tree in trees:
    tree = tree.unrooted()
    for sibling in tree.get_node_matching_name('Greater horseshoe bat').parent.children:
        if sibling.name in ('Microbat', 'Pig', 'Horse'):
            ghb_siblings[sibling.name] += 1
            s_siblings.append(sibling.name)

In [243]:
ghb_siblings

Counter({'Microbat': 450, 'Horse': 241, 'Pig': 187})

In [244]:
ghb_siblings['Microbat']/len(trees)

0.5125284738041003

In [245]:
siblings = list(ghb_siblings.keys())
contingency = pd.DataFrame(np.zeros((3,3)), index=siblings, columns=siblings, dtype=int)
for iqt, neu in zip(s_siblings, ordered_siblings):
    contingency[iqt][neu] += 1

In [246]:
contingency

,Microbat,Horse,Pig
Microbat,227,135,94
Horse,121,47,44
Pig,102,59,49


In [247]:
for i in range(0, len(all_lengths), 4):
    print(s_siblings[i//4], ordered_siblings[i//4])
    edges = get_edges(s_triples[i:i+4], s_lengths[i:i+4], s_losses[i:i+4])
    tips = ['Greater horseshoe bat', 'Microbat', 'Horse', 'Pig']
    affinity = np.zeros((len(tips),)*2)
    for i, tipi in enumerate(tips):
        for j, tipj in enumerate(tips):
            if i == j:
                break
            affinity[i, j] = edges[frozenset((tipi, tipj))]
    affinity += affinity.T
    affinity = pd.DataFrame(affinity, index=tips, columns=tips)
    print(affinity)
    edges = get_edges(all_triples[i:i+4], all_lengths[i:i+4], all_losses[i:i+4])
    tips = ['Greater horseshoe bat', 'Microbat', 'Horse', 'Pig']
    affinity = np.zeros((len(tips),)*2)
    for i, tipi in enumerate(tips):
        for j, tipj in enumerate(tips):
            if i == j:
                break
            affinity[i, j] = edges[frozenset((tipi, tipj))]
    affinity += affinity.T
    affinity = pd.DataFrame(affinity, index=tips, columns=tips)
    print(affinity)
    print()

Microbat Microbat
                       Greater horseshoe bat  Microbat     Horse       Pig
Greater horseshoe bat               0.000000  0.010324  0.002073  0.001224
Microbat                            0.010324  0.000000  0.001676  0.001459
Horse                               0.002073  0.001676  0.000000  0.001459
Pig                                 0.001224  0.001459  0.001459  0.000000
                       Greater horseshoe bat  Microbat     Horse       Pig
Greater horseshoe bat               0.000000  0.063174  0.059007  0.031875
Microbat                            0.063174  0.000000  0.006595  0.005643
Horse                               0.059007  0.006595  0.000000  0.031194
Pig                                 0.031875  0.005643  0.031194  0.000000

Microbat Pig
                       Greater horseshoe bat  Microbat     Horse       Pig
Greater horseshoe bat               0.000000  0.018743  0.015442  0.011878
Microbat                            0.018743  0.000000  0.002172  0.

                       Greater horseshoe bat  Microbat     Horse       Pig
Greater horseshoe bat               0.000000  0.063174  0.059007  0.031875
Microbat                            0.063174  0.000000  0.006595  0.005643
Horse                               0.059007  0.006595  0.000000  0.031194
Pig                                 0.031875  0.005643  0.031194  0.000000

Microbat Microbat
                       Greater horseshoe bat  Microbat     Horse       Pig
Greater horseshoe bat               0.000000  0.002886  0.026069  0.024910
Microbat                            0.002886  0.000000  0.024738  0.028029
Horse                               0.026069  0.024738  0.000000  0.055441
Pig                                 0.024910  0.028029  0.055441  0.000000
                       Greater horseshoe bat  Microbat     Horse       Pig
Greater horseshoe bat               0.000000  0.063174  0.059007  0.031875
Microbat                            0.063174  0.000000  0.006595  0.005643
Horse 

                       Greater horseshoe bat  Microbat     Horse       Pig
Greater horseshoe bat               0.000000  0.015674  0.021125  0.134651
Microbat                            0.015674  0.000000  0.021788  0.125558
Horse                               0.021125  0.021788  0.000000  0.126687
Pig                                 0.134651  0.125558  0.126687  0.000000
                       Greater horseshoe bat  Microbat     Horse       Pig
Greater horseshoe bat               0.000000  0.063174  0.059007  0.031875
Microbat                            0.063174  0.000000  0.006595  0.005643
Horse                               0.059007  0.006595  0.000000  0.031194
Pig                                 0.031875  0.005643  0.031194  0.000000

Horse Pig
                       Greater horseshoe bat  Microbat     Horse       Pig
Greater horseshoe bat               0.000000  0.001880  0.006415  0.005829
Microbat                            0.001880  0.000000  0.001562  0.004218
Horse         

Pig                                 0.031875  0.005643  0.031194  0.000000

Pig Microbat
                       Greater horseshoe bat  Microbat     Horse       Pig
Greater horseshoe bat               0.000000  0.045363  0.042174  0.075331
Microbat                            0.045363  0.000000  0.037035  0.060934
Horse                               0.042174  0.037035  0.000000  0.057141
Pig                                 0.075331  0.060934  0.057141  0.000000
                       Greater horseshoe bat  Microbat     Horse       Pig
Greater horseshoe bat               0.000000  0.063174  0.059007  0.031875
Microbat                            0.063174  0.000000  0.006595  0.005643
Horse                               0.059007  0.006595  0.000000  0.031194
Pig                                 0.031875  0.005643  0.031194  0.000000

Horse Horse
                       Greater horseshoe bat  Microbat     Horse       Pig
Greater horseshoe bat               0.000000  0.001016  0.000882  0.00083

                       Greater horseshoe bat  Microbat     Horse       Pig
Greater horseshoe bat               0.000000  0.001128  0.001193  0.001443
Microbat                            0.001128  0.000000  0.001149  0.001203
Horse                               0.001193  0.001149  0.000000  0.001110
Pig                                 0.001443  0.001203  0.001110  0.000000
                       Greater horseshoe bat  Microbat     Horse       Pig
Greater horseshoe bat               0.000000  0.063174  0.059007  0.031875
Microbat                            0.063174  0.000000  0.006595  0.005643
Horse                               0.059007  0.006595  0.000000  0.031194
Pig                                 0.031875  0.005643  0.031194  0.000000

Horse Horse
                       Greater horseshoe bat  Microbat     Horse       Pig
Greater horseshoe bat               0.000000  0.014853  0.022006  0.027164
Microbat                            0.014853  0.000000  0.011100  0.004709
Horse       

...

Pig                                 0.031875  0.005643  0.031194  0.000000

Microbat Pig
                       Greater horseshoe bat  Microbat     Horse       Pig
Greater horseshoe bat               0.000000  0.099190  0.096222  0.055721
Microbat                            0.099190  0.000000  0.097519  0.066241
Horse                               0.096222  0.097519  0.000000  0.093016
Pig                                 0.055721  0.066241  0.093016  0.000000
                       Greater horseshoe bat  Microbat     Horse       Pig
Greater horseshoe bat               0.000000  0.063174  0.059007  0.031875
Microbat                            0.063174  0.000000  0.006595  0.005643
Horse                               0.059007  0.006595  0.000000  0.031194
Pig                                 0.031875  0.005643  0.031194  0.000000

Microbat Horse
                       Greater horseshoe bat  Microbat     Horse       Pig
Greater horseshoe bat               0.000000  0.107805  0.119143  0.09

                       Greater horseshoe bat  Microbat     Horse       Pig
Greater horseshoe bat               0.000000  0.038400  0.059801  0.011070
Microbat                            0.038400  0.000000  0.051214  0.021796
Horse                               0.059801  0.051214  0.000000  0.066801
Pig                                 0.011070  0.021796  0.066801  0.000000
                       Greater horseshoe bat  Microbat     Horse       Pig
Greater horseshoe bat               0.000000  0.063174  0.059007  0.031875
Microbat                            0.063174  0.000000  0.006595  0.005643
Horse                               0.059007  0.006595  0.000000  0.031194
Pig                                 0.031875  0.005643  0.031194  0.000000

Microbat Microbat
                       Greater horseshoe bat  Microbat     Horse       Pig
Greater horseshoe bat               0.000000  0.082404  0.019557  0.045175
Microbat                            0.082404  0.000000  0.047250  0.108667
Horse 

Pig                                 0.031875  0.005643  0.031194  0.000000

Microbat Microbat
                       Greater horseshoe bat  Microbat     Horse       Pig
Greater horseshoe bat               0.000000  0.031874  0.027609  0.013984
Microbat                            0.031874  0.000000  0.005995  0.002306
Horse                               0.027609  0.005995  0.000000  0.020203
Pig                                 0.013984  0.002306  0.020203  0.000000
                       Greater horseshoe bat  Microbat     Horse       Pig
Greater horseshoe bat               0.000000  0.063174  0.059007  0.031875
Microbat                            0.063174  0.000000  0.006595  0.005643
Horse                               0.059007  0.006595  0.000000  0.031194
Pig                                 0.031875  0.005643  0.031194  0.000000

Pig Microbat
                       Greater horseshoe bat  Microbat     Horse       Pig
Greater horseshoe bat               0.000000  0.002679  0.017220  0

Pig                                 0.031875  0.005643  0.031194  0.000000

Microbat Horse
                       Greater horseshoe bat  Microbat     Horse       Pig
Greater horseshoe bat               0.000000  0.046595  0.030453  0.053159
Microbat                            0.046595  0.000000  0.028513  0.066031
Horse                               0.030453  0.028513  0.000000  0.098698
Pig                                 0.053159  0.066031  0.098698  0.000000
                       Greater horseshoe bat  Microbat     Horse       Pig
Greater horseshoe bat               0.000000  0.063174  0.059007  0.031875
Microbat                            0.063174  0.000000  0.006595  0.005643
Horse                               0.059007  0.006595  0.000000  0.031194
Pig                                 0.031875  0.005643  0.031194  0.000000

Horse Horse
                       Greater horseshoe bat  Microbat     Horse       Pig
Greater horseshoe bat               0.000000  0.104066  0.663482  0.073

Pig                                 0.061647  0.062513  0.112227  0.000000
                       Greater horseshoe bat  Microbat     Horse       Pig
Greater horseshoe bat               0.000000  0.063174  0.059007  0.031875
Microbat                            0.063174  0.000000  0.006595  0.005643
Horse                               0.059007  0.006595  0.000000  0.031194
Pig                                 0.031875  0.005643  0.031194  0.000000

Pig Microbat
                       Greater horseshoe bat  Microbat     Horse       Pig
Greater horseshoe bat               0.000000  0.000591  0.000621  0.000654
Microbat                            0.000591  0.000000  0.000723  0.000694
Horse                               0.000621  0.000723  0.000000  0.000698
Pig                                 0.000654  0.000694  0.000698  0.000000
                       Greater horseshoe bat  Microbat     Horse       Pig
Greater horseshoe bat               0.000000  0.063174  0.059007  0.031875
Microbat   

In [248]:
all_losses

array([[0.005538  , 0.00507691, 0.00486536],
       [0.00349873, 0.00330191, 0.00325043],
       [0.00544869, 0.00480309, 0.00598478],
       ...,
       [0.00748737, 0.00733207, 0.00769578],
       [0.00702603, 0.00656427, 0.00692925],
       [0.00578039, 0.00584953, 0.00573095]])

In [399]:
X = []
for ls, param in zip(all_losses, all_parameters):
    q = param[ls.argmin()]
    q = np.hstack([q[1], q[2,:2], 0])
    q = special.softmax(q)
    print(q)
    X.append(q)
X = np.array(X)

[6.93531930e-03 9.35485894e-01 6.10413148e-04 4.16885480e-02
 8.81038070e-04 1.43987875e-02]
[3.89440912e-04 9.80664292e-01 5.01254035e-04 1.14590139e-02
 6.69189026e-04 6.31681028e-03]
[1.44514826e-03 9.82983319e-01 5.49128458e-04 5.38090006e-03
 6.01439126e-04 9.04006538e-03]
[4.11334621e-04 9.85521733e-01 3.05079380e-04 8.67507183e-03
 4.71630861e-04 4.61515026e-03]
[0.00261761 0.40281319 0.00277307 0.00238438 0.00264218 0.58676957]
[0.00795094 0.808472   0.00323734 0.00242938 0.00162395 0.1762864 ]
[0.00179391 0.39852412 0.00084968 0.0031114  0.01752241 0.57819849]
[3.65548803e-03 9.41544443e-01 4.64516502e-04 1.86884294e-03
 1.57225192e-03 5.08944575e-02]
[2.87898570e-02 9.23856488e-01 5.14611070e-04 7.39095320e-04
 7.62071204e-03 3.84792369e-02]
[9.97278364e-02 8.78215969e-01 3.36613855e-04 6.00490688e-04
 1.18703339e-03 1.99320564e-02]
[0.00095642 0.85913264 0.0021059  0.00391619 0.00400978 0.12987907]
[5.82601556e-03 9.68940296e-01 3.38840877e-04 8.90550382e-04
 1.28495112e-03 

[0.01035078 0.76667466 0.00111928 0.0020758  0.00688113 0.21289835]
[0.02459875 0.76020294 0.0076292  0.00281179 0.03182913 0.1729282 ]
[0.01300186 0.58120511 0.00471193 0.00290564 0.00568598 0.39248948]
[0.03004697 0.54317236 0.00511555 0.00275515 0.03381743 0.38509254]
[0.02386016 0.64472456 0.00900535 0.00265072 0.04026093 0.27949828]
[0.16230434 0.63722834 0.00111283 0.03469968 0.07084086 0.09381396]
[0.00598204 0.49581658 0.00096605 0.03963975 0.0772523  0.38034328]
[0.13817314 0.01032476 0.00268751 0.00488956 0.22056579 0.62335923]
[0.00310692 0.53802339 0.00087841 0.0965422  0.10518201 0.25626707]
[0.00812028 0.07617511 0.00195723 0.00504452 0.08898942 0.81971343]
[1.65699475e-03 9.42071191e-01 5.24599247e-04 9.09434677e-04
 3.54834834e-02 1.93542966e-02]
[0.05060079 0.56362317 0.0021652  0.00309075 0.00391621 0.37660387]
[3.44997652e-03 9.62210611e-01 4.29798799e-04 1.04084199e-03
 1.45601862e-02 1.83085850e-02]
[1.22185003e-03 9.09005890e-01 7.96128771e-04 1.98567749e-03
 5.27

[0.01661648 0.68992674 0.00173921 0.00167083 0.01068425 0.2793625 ]
[0.01618299 0.08615941 0.00420146 0.00800702 0.26194347 0.62350565]
[0.17972117 0.6582891  0.01287422 0.00143655 0.00824028 0.13943868]
[1.84779668e-01 7.26332434e-01 2.35705642e-02 3.96904528e-04
 1.32752497e-02 5.16451791e-02]
[2.29831877e-01 6.92204485e-01 1.77446489e-03 4.40343986e-04
 3.23383290e-03 7.25149967e-02]
[2.43948421e-01 6.58582947e-01 1.01314936e-03 4.42356874e-04
 2.25406405e-03 9.37590617e-02]
[0.11080633 0.69340606 0.00801493 0.0045556  0.05553409 0.12768299]
[0.09234354 0.65464493 0.01105155 0.01374623 0.08544619 0.14276755]
[0.17962628 0.57251684 0.00724964 0.01303857 0.05667905 0.17088962]
[0.10560445 0.67637599 0.02845686 0.0013778  0.06099467 0.12719024]
[0.00670393 0.23104417 0.00199409 0.00432042 0.00411034 0.75182705]
[0.0023282  0.57146803 0.00142986 0.00193191 0.00144769 0.42139431]
[6.50848806e-04 9.07873264e-01 4.94268036e-04 7.53076733e-04
 5.69556966e-04 8.96589852e-02]
[0.00386663 0.44

[0.0357841  0.06391819 0.00446609 0.05672541 0.00412374 0.83498248]
[0.0078176  0.83019923 0.00088339 0.00162185 0.01819627 0.14128165]
[0.00203206 0.9131642  0.00352806 0.00360848 0.0028552  0.074812  ]
[0.00174756 0.89251049 0.00270215 0.00338541 0.00400373 0.09565065]
[0.01146908 0.66973232 0.01750072 0.02313413 0.00270047 0.27546327]
[0.00978505 0.91641157 0.00895936 0.01871849 0.00199205 0.04413348]
[2.70499095e-03 9.22195214e-01 8.83302253e-04 5.48008750e-03
 2.35650387e-04 6.85007548e-02]
[2.16796027e-03 8.56793904e-01 8.06834598e-04 8.24326244e-03
 4.13178883e-03 1.27856250e-01]
[2.17773248e-03 9.08564753e-01 3.37745880e-04 1.81549314e-02
 2.08775603e-03 6.86770809e-02]
[3.91342894e-03 9.23084134e-01 1.96502668e-04 1.42861329e-02
 6.47261450e-04 5.78725405e-02]
[0.03086434 0.57430714 0.0271205  0.00104187 0.00170675 0.36495941]
[0.03940532 0.52740973 0.00306829 0.00318746 0.00215739 0.42477181]
[0.05227788 0.46489406 0.01722186 0.00179282 0.0005669  0.46324648]
[0.03374419 0.49

In [400]:
from sklearn.cluster import KMeans, AffinityPropagation
from sklearn.metrics import davies_bouldin_score
fit = AffinityPropagation().fit(X.T)
davies_bouldin_score(X.T, fit.labels_)

/home/ben/miniconda3/envs/tensorflow/lib/python3.8/site-packages/sklearn/cluster/_affinity_propagation.py:148: FutureWarning: 'random_state' has been introduced in 0.23. It will be set to None starting from 1.0 (renaming of 0.25) which means that results will differ at every function call. Set 'random_state' to None to silence this warning, or to 0 to keep the behavior of versions <0.23.
  warnings.warn(


0.10026715002320236

In [401]:
fit.labels_

array([1, 0, 1, 1, 1, 2])

In [263]:
X.shape

(3512, 6)

In [306]:
AffinityPropagation?